In [0]:
# Caminho do arquivo Titanic
file_path = "dbfs:/FileStore/tables/titanic_data.csv"

# Carregar o arquivo CSV no PySpark
titanic_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Exibir as primeiras linhas e o esquema do DataFrame
titanic_df.show(10)  # Visualizar os dados
titanic_df.printSchema()  # Ver o esquema

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

# Exclusão de Colunas e Redefinição de Survived

In [0]:
from pyspark.sql.functions import when

# Transformar `Survived` e remover colunas desnecessárias
titanic_df = (
    titanic_df.drop("PassengerId", "Ticket")
    .withColumn("Survived", when(titanic_df.Survived == 1, "Sobreviveu").otherwise("Não_Sobreviveu"))
)

# Visualizar as mudanças
titanic_df.show(10)

+--------------+------+--------------------+------+----+-----+-----+-------+-----+--------+
|      Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Fare|Cabin|Embarked|
+--------------+------+--------------------+------+----+-----+-----+-------+-----+--------+
|Não_Sobreviveu|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|   7.25| null|       S|
|    Sobreviveu|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|71.2833|  C85|       C|
|    Sobreviveu|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|  7.925| null|       S|
|    Sobreviveu|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|   53.1| C123|       S|
|Não_Sobreviveu|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|   8.05| null|       S|
|Não_Sobreviveu|     3|    Moran, Mr. James|  male|null|    0|    0| 8.4583| null|       Q|
|Não_Sobreviveu|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|51.8625|  E46|       S|
|Não_Sobreviveu|     3|Palsson, Master. ...|  male| 2.0|    3|    1| 21.075| nul

# Verificação de Valores Ausentes

In [0]:
from pyspark.sql.functions import col, sum

# Contar valores ausentes em cada coluna
missing_values = (
    titanic_df.select([(sum(col(c).isNull().cast("int")).alias(c)) for c in titanic_df.columns])
)

# Exibir o total de valores ausentes por coluna
missing_values.show()

+--------+------+----+---+---+-----+-----+----+-----+--------+
|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Fare|Cabin|Embarked|
+--------+------+----+---+---+-----+-----+----+-----+--------+
|       0|     0|   0|  0|177|    0|    0|   0|  687|       2|
+--------+------+----+---+---+-----+-----+----+-----+--------+



# Tratamento de Valores Ausentes

In [0]:
from pyspark.sql.functions import mean, lit, expr

# Tratamento da coluna Age
mean_age = titanic_df.select(mean(col("Age"))).first()[0]
titanic_df = titanic_df.fillna({"Age": mean_age})

# Tratamento da coluna Cabin
titanic_df = titanic_df.fillna({"Cabin": "Unknown"})

# Tratamento da coluna Embarked
mode_embarked = titanic_df.groupBy("Embarked").count().orderBy("count", ascending=False).first()[0]
titanic_df = titanic_df.fillna({"Embarked": mode_embarked})

# Verificar novamente os valores ausentes
titanic_df.select([(sum(col(c).isNull().cast("int")).alias(c)) for c in titanic_df.columns]).show()

+--------+------+----+---+---+-----+-----+----+-----+--------+
|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Fare|Cabin|Embarked|
+--------+------+----+---+---+-----+-----+----+-----+--------+
|       0|     0|   0|  0|  0|    0|    0|   0|    0|       0|
+--------+------+----+---+---+-----+-----+----+-----+--------+



# Estatísticas Descritivas

In [0]:
from pyspark.sql.functions import col, skewness, kurtosis, sum as sum_, stddev
from pyspark.sql.types import IntegerType, DoubleType

# Calcular estatísticas descritivas para todas as colunas numéricas
descriptive_stats = titanic_df.describe()

# Exibir as estatísticas principais
descriptive_stats.show()

# Calcular skewness e kurtosis para as colunas numéricas
skewness_values = titanic_df.select([skewness(c).alias(c) for c in titanic_df.columns if isinstance(titanic_df.schema[c].dataType, (IntegerType, DoubleType))])
kurtosis_values = titanic_df.select([kurtosis(c).alias(c) for c in titanic_df.columns if isinstance(titanic_df.schema[c].dataType, (IntegerType, DoubleType))])

# Exibir os resultados de skewness e kurtosis
print("Skewness:")
skewness_values.show()

print("Kurtosis:")
kurtosis_values.show()

# Calcular sum e stddev para as colunas numéricas
sum_values = titanic_df.select([sum_(c).alias(c) for c in titanic_df.columns if isinstance(titanic_df.schema[c].dataType, (IntegerType, DoubleType))])
stddev_values = titanic_df.select([stddev(c).alias(c) for c in titanic_df.columns if isinstance(titanic_df.schema[c].dataType, (IntegerType, DoubleType))])

# Exibir os resultados de sum e stddev
print("Sum:")
sum_values.show()

print("Standard Deviation (std):")
stddev_values.show()

+-------+--------------+------------------+--------------------+------+------------------+------------------+-------------------+-----------------+-------+--------+----------+
|summary|      Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|             Fare|  Cabin|Embarked|Fare_Range|
+-------+--------------+------------------+--------------------+------+------------------+------------------+-------------------+-----------------+-------+--------+----------+
|  count|           891|               891|                 891|   891|               891|               891|                891|              891|    891|     891|       891|
|   mean|          null| 2.308641975308642|                null|  null|29.699117647058763|0.5230078563411896|0.38159371492704824| 32.2042079685746|   null|    null|      null|
| stddev|          null|0.8360712409770491|                null|  null|13.002015226002891|1.1027434322934315| 0.80605722

# Matriz de Correlação

In [0]:
pandas_df = titanic_df.toPandas()

# Converter temporariamente 'Survived' de volta para valores numéricos
pandas_df["Survived"] = pandas_df["Survived"].map({"Sobreviveu": 1, "Não_Sobreviveu": 0})

# Calcular a matriz de correlação
correlation_matrix = pandas_df.corr()

# Exibir a matriz de correlação
print("Matriz de Correlação:")
print(correlation_matrix)

# Identificar correlação com 'Survived'
print("\nCorrelação com Survived:")
print(correlation_matrix["Survived"].sort_values(ascending=False))

Matriz de Correlação:
          Survived    Pclass       Age     SibSp     Parch      Fare
Survived  1.000000 -0.338481 -0.069809 -0.035322  0.081629  0.257307
Pclass   -0.338481  1.000000 -0.331339  0.083081  0.018443 -0.549500
Age      -0.069809 -0.331339  1.000000 -0.232625 -0.179191  0.091566
SibSp    -0.035322  0.083081 -0.232625  1.000000  0.414838  0.159651
Parch     0.081629  0.018443 -0.179191  0.414838  1.000000  0.216225
Fare      0.257307 -0.549500  0.091566  0.159651  0.216225  1.000000

Correlação com Survived:
Survived    1.000000
Fare        0.257307
Parch       0.081629
SibSp      -0.035322
Age        -0.069809
Pclass     -0.338481
Name: Survived, dtype: float64


# Porcentagem de Sobrevivência por Sexo

In [0]:
# Calcular o total de sobreviventes por sexo
total_por_sexo = titanic_df.groupBy("Sex").count().withColumnRenamed("count", "Total")
sobreviventes_por_sexo = titanic_df.filter(titanic_df["Survived"] == "Sobreviveu") \
                                   .groupBy("Sex").count().withColumnRenamed("count", "Sobreviventes")

# Juntar os resultados
percentual_sobrevivencia = total_por_sexo.join(sobreviventes_por_sexo, "Sex") \
                                         .withColumn("Percentual_Sobrevivencia", 
                                                     (sobreviventes_por_sexo["Sobreviventes"] / total_por_sexo["Total"]) * 100)

percentual_sobrevivencia.show()


+------+-----+-------------+------------------------+
|   Sex|Total|Sobreviventes|Percentual_Sobrevivencia|
+------+-----+-------------+------------------------+
|female|  314|          233|       74.20382165605095|
|  male|  577|          109|      18.890814558058924|
+------+-----+-------------+------------------------+



# Média do Valor Pago por Sexo

In [0]:
# Calcular a média de 'Fare' por sexo
media_fare_por_sexo = titanic_df.groupBy("Sex").agg({"Fare": "mean"}) \
                                .withColumnRenamed("avg(Fare)", "Media_Fare")

media_fare_por_sexo.show()

+------+------------------+
|   Sex|        Media_Fare|
+------+------------------+
|female| 44.47981783439487|
|  male|25.523893414211418|
+------+------------------+



# Média do Valor Pago por Classe

In [0]:
# Calcular a média de 'Fare' por classe (Pclass)
media_fare_por_classe = titanic_df.groupBy("Pclass").agg({"Fare": "mean"}) \
                                  .withColumnRenamed("avg(Fare)", "Media_Fare") \
                                   .orderBy("Media_Fare", ascending=False)

media_fare_por_classe.show()

+------+------------------+
|Pclass|        Media_Fare|
+------+------------------+
|     1| 84.15468749999992|
|     2| 20.66218315217391|
|     3|13.675550101832997|
+------+------------------+



# Média do Valor Pago por Local de Embarque

In [0]:
# Calcular a média de 'Fare' por local de embarque (Embarked)
media_fare_por_embarque = titanic_df.groupBy("Embarked").agg({"Fare": "mean"}) \
                                    .withColumnRenamed("avg(Fare)", "Media_Fare") \
                                     .orderBy("Media_Fare", ascending=False)

media_fare_por_embarque.show()

+--------+------------------+
|Embarked|        Media_Fare|
+--------+------------------+
|       C| 59.95414404761905|
|       S|27.243651393188795|
|       Q|13.276029870129872|
+--------+------------------+



# Contagem de Passageiros por Local de Embarque

In [0]:
# Contar passageiros por local de embarque
contagem_embarque = titanic_df.groupBy("Embarked").count().withColumnRenamed("count", "Total_Passageiros")

contagem_embarque.show()

+--------+-----------------+
|Embarked|Total_Passageiros|
+--------+-----------------+
|       Q|               77|
|       C|              168|
|       S|              646|
+--------+-----------------+



# Análise de Tarifa Média por Embarque, Sexo e Classe

In [0]:
# Agrupando por 'Embarked', 'Sex' e 'Pclass' para calcular a contagem e a média de 'Fare'
from pyspark.sql.functions import count, mean

analise_fare = (
    titanic_df.groupBy("Embarked", "Sex", "Pclass")
    .agg(
        count("Fare").alias("Count_Fare"),
        mean("Fare").alias("Mean_Fare")
    )
    .orderBy("Embarked", "Sex", "Pclass")
)

analise_fare.show()

+--------+------+------+----------+------------------+
|Embarked|   Sex|Pclass|Count_Fare|         Mean_Fare|
+--------+------+------+----------+------------------+
|       C|female|     1|        43| 115.6403093023256|
|       C|female|     2|         7|25.268457142857148|
|       C|female|     3|        23|14.694926086956524|
|       C|  male|     1|        42| 93.53670714285715|
|       C|  male|     2|        10|25.421249999999997|
|       C|  male|     3|        43| 9.352237209302325|
|       Q|female|     1|         1|              90.0|
|       Q|female|     2|         2|             12.35|
|       Q|female|     3|        33|10.307833333333331|
|       Q|  male|     1|         1|              90.0|
|       Q|  male|     2|         1|             12.35|
|       Q|  male|     3|        39| 11.92425128205128|
|       S|female|     1|        50| 98.26583399999998|
|       S|female|     2|        67| 21.91268656716418|
|       S|female|     3|        88| 18.67007727272727|
|       S|

# Outliers em Fare

In [0]:
# Calcular os quartis e o intervalo interquartil (IQR)
fare_stats = titanic_df.selectExpr("percentile_approx(Fare, 0.25) as Q1", 
                                   "percentile_approx(Fare, 0.75) as Q3").collect()
Q1 = fare_stats[0]["Q1"]
Q3 = fare_stats[0]["Q3"]
IQR = Q3 - Q1

# Limites para outliers
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

# Identificar valores fora do intervalo
outliers_fare = titanic_df.filter((titanic_df["Fare"] < lower_limit) | (titanic_df["Fare"] > upper_limit))
print(f"Outliers identificados: {outliers_fare.count()}")

outliers_fare.show()

Outliers identificados: 116
+--------------+------+--------------------+------+-----------------+-----+-----+--------+-----------+--------+
|      Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|    Fare|      Cabin|Embarked|
+--------------+------+--------------------+------+-----------------+-----+-----+--------+-----------+--------+
|    Sobreviveu|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0| 71.2833|        C85|       C|
|Não_Sobreviveu|     1|Fortune, Mr. Char...|  male|             19.0|    3|    2|   263.0|C23 C25 C27|       S|
|    Sobreviveu|     1|Spencer, Mrs. Wil...|female|29.69911764705882|    1|    0|146.5208|        B78|       C|
|Não_Sobreviveu|     1|Meyer, Mr. Edgar ...|  male|             28.0|    1|    0| 82.1708|    Unknown|       C|
|    Sobreviveu|     1|Harper, Mrs. Henr...|female|             49.0|    1|    0| 76.7292|        D33|       C|
|    Sobreviveu|     1| Icard, Miss. Amelie|female|             38.0|    0| 

# Categorizar Fare em Faixas

In [0]:
# Criar categorias para 'Fare'
from pyspark.sql.functions import when

titanic_df = titanic_df.withColumn(
    "Fare_Range",
    when(titanic_df["Fare"] < Q1, "Baixo")
    .when(titanic_df["Fare"] > Q3, "Alto")
    .otherwise("Medio")
)

titanic_df.groupBy("Fare_Range").count().show()

+----------+-----+
|Fare_Range|count|
+----------+-----+
|     Baixo|  185|
|     Medio|  484|
|      Alto|  222|
+----------+-----+

